In [26]:
# https://www.kaggle.com/gulsahdemiryurek/harry-potter-dataset

In [27]:
from pathlib import Path
from collections import Counter
from textblob import TextBlob
import nltk
import os

nltk.download('brown')
nltk.download('punkt')

root_dir = Path(os.path.abspath('__file__')).parent
dataset_dir = root_dir / 'data' / 'book'
book1_dir = dataset_dir / 'Harry Potter 1.csv'
book2_dir = dataset_dir / 'Harry Potter 2.csv'
book3_dir = dataset_dir / 'Harry Potter 3.csv'

print(root_dir)
print(dataset_dir)

/home/shane/workspace/sna/potter
/home/shane/workspace/sna/potter/data/book


[nltk_data] Downloading package brown to /home/shane/nltk_data...
[nltk_data]   Package brown is already up-to-date!
[nltk_data] Downloading package punkt to /home/shane/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


In [28]:
from nltk.sentiment.vader import SentimentIntensityAnalyzer

analyser = SentimentIntensityAnalyzer()

with open(book3_dir, 'r') as f:
    book = list(f.readlines())[1:]
    book = list(map(lambda x: x.split(';'), book))
    book = [list(map(lambda x: x.strip(), x)) for x in book]

In [29]:
characters = []
nouns = []
character_nouns = {}

for character, sentence in book:
    characters.append(character.title())
    blob = TextBlob(sentence)
    n = blob.noun_phrases
    n = list(map(lambda x: x.lower(), n))
    nouns.extend(n)

    if character.title() in character_nouns:
        character_nouns[character.title()].extend(n)
    else:
        character_nouns[character.title()] = list(n)

characters = list(dict(sorted(dict(Counter(characters)).items(), key=lambda x: x[1], reverse=True)).keys())
nouns = list(dict(sorted(dict(Counter(nouns)).items(), key=lambda x: x[1], reverse=True)).keys())

print("Total characters")
print(len(characters))
print()

print("Total nouns")
print(len(nouns))

character_register = dict(zip(characters, range(1, len(characters)+1)))
noun_register = dict(zip(nouns, range(len(characters)+1, len(characters)+len(nouns)+1)))

for k in character_nouns:
    character_nouns[k] = dict(sorted(dict(Counter(character_nouns[k])).items(), key=lambda x: x[1], reverse=True))

Total characters
50

Total nouns
567


In [30]:
with open('book3_character_noun_bipartite.net', 'w') as f:
    f.write(f"*Vertices\t{list(noun_register.values())[-1]}\t{list(character_register.values())[-1]}\n")
    for name, i in character_register.items():
        f.write(f"{i}\t\"{name}\"\n")
    for noun, i in noun_register.items():
        f.write(f"{i}\t\"{noun}\"\n")
    f.write(f"*Edges\n")
    for character, d in character_nouns.items():
        for k, v in d.items():
            f.write(f"{character_register[character]}\t{noun_register[k]}\t{v}\n")